<a href="https://colab.research.google.com/github/cawllol/NLP/blob/master/IMDB_MoviesSentimentAnalysisUsingBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ktrain

In [2]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import ktrain
from ktrain import text

In [7]:
data_train=pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx',dtype=str)
data_train.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [6]:
data_test=pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx',dtype=str)
data_test.head()

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg


In [9]:
(X_train,y_train),(X_test,y_test),preprocess=text.texts_from_df(train_df=data_train,
                                     text_column='Reviews',
                                     label_columns='Sentiment',
                                     val_df=data_test,
                                     maxlen=400,
                                     preprocess_mode='bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [10]:
X_train[0].shape

(25000, 400)

In [13]:
model=text.text_classifier(name='bert',
                           train_data=(X_train,y_train),   preproc=preprocess)                                                   

Is Multi-Label? False
maxlen is 400
done.


In [14]:
#get learning rate
learner=ktrain.get_learner(model=model,
                           train_data=(X_train,y_train),
                           val_data=(X_test,y_test),
                           batch_size=6)

In [ ]:
#getting optimal learning rate
#learner.lr_find()
#learner.lr_plot()

#optimal learning rate for this model is 2e-5

In [ ]:
learner.fit_onecycle(lr=2e-5,epochs=1)

In [15]:
learner

In [16]:
#prediction
predictor=ktrain.get_predictor(learner.model,preprocess)

In [29]:
data=['this movie was horrible. the plot was really boring. Acting was okay. though',
      'the film really sucked. i want my money back']

In [30]:
predictor.predict(data)

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:229: UserWarning: List or array of two texts supplied, so task being treated as text classification. If this is a sentence pair classification task, please cast to tuple.
  'If this is a sentence pair classification task, please cast to tuple.')


['neg', 'neg']

In [ ]:
predictor.save('/content/bert')